In [1]:
import pandas as pd

from dbcp.helpers import get_sql_engine

engine = get_sql_engine()

with engine.connect() as con:
    eia860m = pd.read_sql_table("pudl_eia860m_changelog", con, schema="data_warehouse")
    pudl_eia860m_status_codes = pd.read_sql_table("pudl_eia860m_status_codes", con, schema="data_warehouse")

In [2]:
PREVIOUS_QUARTER_DATE = "2024-07-01"

def grab_current_quarter(df):
    recent_quarter = df.loc[df.groupby(["generator_id", "plant_id_eia"])['report_date'].idxmax()]
    assert ~recent_quarter.duplicated(subset=["generator_id", "plant_id_eia"]).any()
    return recent_quarter

def grab_previous_quarter(df):
    previous_quarter = df[df.report_date < PREVIOUS_QUARTER_DATE]
    previous_quarter = previous_quarter.loc[previous_quarter.groupby(["generator_id", "plant_id_eia"])['report_date'].idxmax()]
    assert ~previous_quarter.duplicated(subset=["generator_id", "plant_id_eia"]).any()
    return previous_quarter


def pct_change_mw_by_status(df):
    recent_quarter = grab_current_quarter(df)
    recent_quarter_mw_by_status = recent_quarter.groupby("operational_status_code").capacity_mw.sum()

    previous_quarter = grab_previous_quarter(df)
    
    previous_quarter_mw_by_status = previous_quarter.groupby("operational_status_code").capacity_mw.sum()

    return ((recent_quarter_mw_by_status - previous_quarter_mw_by_status) / previous_quarter_mw_by_status) * 100
    

How many generators had a status change in this quarter update? We shouldn't expect that many generators to have status changes.

Merge the quarters together using the generator ID. Each quarter should only have one record for each generator so the merge should be one to one.


In [3]:
previous_quarter = grab_previous_quarter(eia860m)
current_quarter = grab_current_quarter(eia860m)

In [4]:
merged_quarters = previous_quarter.merge(current_quarter, on=["generator_id", "plant_id_eia"], validate="1:1", suffixes=("_previous", "_current"))

different_status_codes = merged_quarters["operational_status_code_previous"].ne(merged_quarters["operational_status_code_current"])
different_status_codes.value_counts()


False    35549
True       466
dtype: int64

For the generators have have a different status in the new update, check to see if the status change makes sense: ("Operational to Retired", "Under Construction to Operational", etc). A highlevel check to make sure the status changes make sense is to see if the status code numbers stay the same or increase. Higher number operational codes represent more advanced stages in a generator's life cycle.

In [5]:
new_status_code_is_greater = merged_quarters["operational_status_code_previous"].le(merged_quarters["operational_status_code_current"])

new_status_code_is_greater.value_counts()

True     36009
False        6
dtype: int64

In [6]:
merged_quarters[~new_status_code_is_greater][["raw_operational_status_code_previous", "raw_operational_status_code_current"]]

,raw_operational_status_code_previous,raw_operational_status_code_current
17994,RE,OS
18021,RE,OS
18045,RE,OS
22228,RE,OP
28916,RE,OP
31142,T,L


Looks like there are a handful of generators that came out of retirement. Let dig into the status codes of the generators that have a new status in the udpated data.

In [7]:
pd.set_option('display.max_colwidth', None)

pudl_eia860m_status_codes

,code,status,description
0,OT,99,proposed
1,IP,98,"Planned new indefinitely postponed, or no longer in resource plan"
2,P,1,Planned for installation but regulatory approvals not initiated; Not under construction
3,L,2,Not under construction but site preparation could be underway
4,T,3,Regulatory approvals received. Not under construction but site preparation could be underway
5,U,4,"Under construction, less than or equal to 50 percent complete (based on construction time to date of operation)"
6,V,5,"Under construction, more than 50 percent complete (based on construction time to date of operation)"
7,TS,6,"Construction complete, but not yet in commercial operation (including low power testing of nuclear units). Operating under test conditions."
8,OA,7,Was not used for some or all of the reporting period but is expected to be returned to service in the next calendar year.
9,OP,7,"Operating (in commercial service or out of service within 365 days). For generators, this means in service (commercial operation) and producing some electricity. Includes peaking units that are run on an as needed (intermittent or seasonal) basis."


In [8]:
merged_quarters[different_status_codes][["raw_operational_status_code_previous", "raw_operational_status_code_current"]].value_counts()

raw_operational_status_code_previous  raw_operational_status_code_current
OP                                    RE                                     85
U                                     V                                      78
V                                     OP                                     69
TS                                    OP                                     53
OS                                    RE                                     34
V                                     TS                                     26
T                                     U                                      21
                                      V                                      15
U                                     OP                                     12
L                                     U                                      10
P                                     V                                       8
L                                     T       

Look at capacity change for each status code.

In [9]:
pct_change_mw_by_status(eia860m)

operational_status_code
1     10.430908
2      6.532631
3      4.905398
4     -5.174416
5      9.569163
6     12.365107
7      0.714077
8      0.079723
99     0.000000
Name: capacity_mw, dtype: float64

## Capacity by status by ISO

In [10]:
ISO_REGIONS = ("MISO", "PJM", "CISO", "ERCO", "ISNE", "NYIS", "SWPP") 

In [11]:
eia860_isos = eia860m[eia860m.balancing_authority_code_eia.isin(ISO_REGIONS)]


for region in ISO_REGIONS:
    print(region)
    pct_change = pct_change_mw_by_status(eia860_isos[eia860_isos["balancing_authority_code_eia"] == region])
    print(pct_change)
    print()

MISO
operational_status_code
1     0.848435
2    22.710135
3    13.838777
4    10.811155
5    18.757170
6    15.527606
7     0.771333
8     0.050665
Name: capacity_mw, dtype: float64

PJM
operational_status_code
1       9.898398
2       1.209212
3      -0.003964
4       1.255510
5      -0.968126
6     209.016394
7       0.163477
8       0.101088
99      0.000000
Name: capacity_mw, dtype: float64

CISO
operational_status_code
1      7.932810
2     -0.425961
3     25.064151
4     -8.198664
5      3.443937
6      0.943523
7      1.891107
8     -0.076096
99     0.000000
Name: capacity_mw, dtype: float64

ERCO
operational_status_code
1      9.219364
2      9.677581
3      8.698993
4     -1.511497
5    -19.986722
6      6.414715
7      2.277254
8      0.000000
99     0.000000
Name: capacity_mw, dtype: float64

ISNE
operational_status_code
1       0.000000
2     -37.193092
3      -1.548947
4     189.530504
5       1.449929
6     -82.614942
7       1.141464
8      -1.600243
99      0.000000
Na